# Processing the final dataset



In [1]:
import ee
import geemap

# Authenticate to Earth Engine
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

data_folder = "projects/amazon-forest-regrowth/assets"



In [2]:
# 1 = Amazonia
# 2 = Caatinga
# 3 = Cerrado
# 4 = Mata Atlantica
# 5 = Pampa
# 6 = Pantanal

# region = "countrywide"
# region = "amaz" #for Amazonia
region = "atl" #for Mata Atlantica
# region = "cerr" #for Cerrado

biomes_br = ee.FeatureCollection(f"{data_folder}/raw/biomes_br")

if region == "countrywide":
    # Countrywide
    roi = biomes_br.geometry().dissolve()
elif region == "amaz":
    # Amazonia = 1
    roi = biomes_br.filter(ee.Filter.eq("CD_Bioma", 1))
elif region == "cerr":
    roi = biomes_br.filter(ee.Filter.eq("CD_Bioma", 3))
elif region == "atl":
    roi = biomes_br.filter(ee.Filter.eq("CD_Bioma", 4))



In [8]:
mature_biomass = ee.Image(f"{data_folder}/mature_biomass_500m_countrywide")#.clip(roi)
mature_biomass_vector = mature_biomass.reduceToVectors(geometry=roi, scale=500, maxPixels=1e13)
proj = mature_biomass.projection()

secondary = ee.Image(f"{data_folder}/age_agbd").select("age")
# distances_image = mature_biomass_vector.distance(searchRadius = 500000).toInt().updateMask(secondary)


# # Export the mean_biomass_image as an asset
# task = ee.batch.Export.image.toAsset(
#     image=distances_image,
#     assetId = "projects/amazon-forest-regrowth/assets/distance_to_mature_biomass",
#     description="distance_to_mature_biomass",
#     region=roi,
#     scale=500,
#     crs=proj.crs(),
#     crsTransform=proj.transform(),
#     maxPixels=1e13
# )
# task.start()

In [9]:
distance = ee.Image(f"{data_folder}/distance_to_mature_biomass").clip(roi)

distances_vector = mature_biomass.sample(region = roi, scale = 500, numPixels = 10)
    #proj.nominalScale(), projection = proj.crs(),
    #, geometries = True
# )


# Export the mean_biomass_image as an asset
task = ee.batch.Export.table.toAsset(
    collection=distances_vector,
    assetId = "projects/amazon-forest-regrowth/assets/distances_vector_sample",
    description="distances_vector_sample"
)

task.start()


In [6]:

distances_vector = distance.stratifiedSample(
    numPoints = 50, region = roi.geometry(), geometries = True
)

# Export the mean_biomass_image as an asset
task = ee.batch.Export.table.toAsset(
    collection=distances_vector,
    assetId = "projects/amazon-forest-regrowth/assets/distances_vector",
    description="distances_vector"
)

task.start()


In [5]:
# distance = ee.Image(f"{data_folder}/distance_to_mature_biomass").clip(roi)

# map = geemap.Map()
# map.addLayer(roi, {}, "roi")
# map.addLayer(distance, {"min": 0, "max": 10000}, "distance")
# map.addLayer(distances_vector, {}, "distances_vector")
# map

In [ ]:
# Function to buffer and calculate mean
def buffer_and_calculate_mean(f):
    distance = ee.Number(f.get('distance'))
    buffered = ee.Feature(f.buffer(distance.add(2500)))
    return buffered

distances_vector_buffer = distances_vector.map(buffer_and_calculate_mean)







In [54]:

nearest_biomass = mature_biomass.reduceRegions(distances_vector_buffer, ee.Reducer.firstNonNull(), 
                                          crs = proj['crs'], crsTransform = proj['transform'])

# Export the mean_biomass_image as an asset
task = ee.batch.Export.table.toAsset(
    collection=nearest_biomass,
    assetId = "projects/amazon-forest-regrowth/assets/nearest_biomass",
    description="nearest_biomass"
)

task.start()


In [24]:
mature_biomass = ee.Image(f"{data_folder}/mature_biomass_500m_countrywide").clip(roi)

# def export_csv(year):
year = 10
if (year == 0):
    land_use_years = ee.Image(f"{data_folder}/land_use")
    name = f"{region}_all"
else:
    land_use_years = ee.Image(f"{data_folder}/land_use_{year}_years")
    name = f"{region}_{year}yr"

age_agbd = ee.Image(f"{data_folder}/age_agbd")
categorical = ee.Image(f"{data_folder}/categorical")
yearly_mean_prec = ee.Image(f"{data_folder}/yearly_mean_prec")
yearly_SI = ee.Image(f"{data_folder}/yearly_SI")
cwd = ee.Image(f"{data_folder}/raw/cwd_chave")
sur_cover = ee.Image(f"{data_folder}/sur_cover").rename("sur_cover")
one_hectare_mask = ee.Image(f"{data_folder}/one_hectare_mask")
distance = ee.Image(f"{data_folder}/distance_to_mature_biomass")

combined_mask = one_hectare_mask.And(land_use_years.select("last_LU"))

unified_data = (
    age_agbd.addBands(
        [distance, categorical, land_use_years, cwd, sur_cover, yearly_mean_prec, yearly_SI
    ]).updateMask(combined_mask)
)

unified_data_fc = unified_data.stratifiedSample(
    numPoints = 10000, classBand = 'biome', region = roi.geometry(), geometries = True
)

pts_bm = unified_data_fc.map(buffer_and_calculate_mean)

task = ee.batch.Export.table.toDrive(
    collection=pts_bm, description=name
)
task.start()

task = ee.batch.Export.table.toAsset(
    collection=pts_bm, description=name, assetId=f"projects/amazon-forest-regrowth/assets/{name}"
)
task.start()

# export_csv(0)
# export_csv(5)
# export_csv(10)
# export_csv(15)



In [37]:
# mature_biomass = ee.Image(f"{data_folder}/mature_biomass")
# mature_biomass_500m_countrywide = ee.Image(f"{data_folder}/mature_biomass_500m_countrywide")
# biomass_all = ee.Image(f"{data_folder}/raw/biomass")


# map = geemap.Map()
# map.addLayer(biomass_all, {"min": 0, "max": 400, "palette": ["blue", "red"]}, 'biomass_all')
# map.addLayer(mature_biomass, {"min": 0, "max": 400, "palette": ["blue", "red"]}, 'mature_biomass')
# map.addLayer(mature_biomass_100m, {"min": 0, "max": 400, "palette": ["blue", "red"]}, 'mature_biomass_100m')
# map

In [53]:
gaul = ee.FeatureCollection('FAO/GAUL/2015/level0')

# List of South American country names
south_american_countries = [
    'Argentina', 'Bolivia', 'Brazil', 'Chile', 'Colombia', 'Ecuador',
    'French Guiana', 'Guyana', 'Paraguay', 'Peru', 'Suriname', 'Uruguay', 'Venezuela'
]

# Filter the GAUL dataset for South American countries using ADM0_NAME
south_america = gaul.filter(ee.Filter.inList('ADM0_NAME', south_american_countries))
# # Merge all South American features into a single feature
south_america_merged = south_america.union().geometry()

# Extract Brazil separately
brazil = gaul.filter(ee.Filter.eq('ADM0_NAME', 'Brazil')).first().geometry()


# edge_mask = brazil_buffer.difference(south_america_merged.geometry())
# secondary_notedge = secondary.updateMask(secondary.mask(brazil_buffer))

map = geemap.Map()
map.addLayer(south_america_merged, {}, "tst")
map

# Create a mask by subtracting the buffered area from the South America feature

# # Export the mean_biomass_image as an asset
# task = ee.batch.Export.table.toAsset(
#     collection=brazil_buffer,
#     assetId = "projects/amazon-forest-regrowth/assets/brazil_buffer",
#     description="brazil_buffer"
# )
# task.start()

KeyboardInterrupt: 